In [ ]:
import pandas as pd
from sqlalchemy import (
    create_engine, Table, Column, Integer, String, Float, Numeric, Date,
    ForeignKey, func, MetaData, CheckConstraint
)
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
from datetime import datetime
import os
import logging

logger = logging.getLogger(__name__)

# =========================================================================
# CONFIGURAÇÕES DE BANCO DE DADOS
# =========================================================================

DATABASE_CONFIG = {
    'username': os.getenv('DB_USERNAME', 'postgres'),
    'password': os.getenv('DB_PASSWORD', 'U7urkInVDg[(D^{&'),  # Ajuste conforme necessário
    'host': os.getenv('DB_HOST', '34.130.95.218'),
    'port': os.getenv('DB_PORT', '5432'),
    'database': os.getenv('DB_NAME', 'postgres'),
}

def get_database_url(config):
    return f"postgresql://{config['username']}:{config['password']}@" \
           f"{config['host']}:{config['port']}/{config['database']}"

DATABASE_URL = get_database_url(DATABASE_CONFIG)
engine = create_engine(DATABASE_URL, echo=False)
metadata = MetaData()

# =========================================================================
# DEFINIÇÃO DAS TABELAS
# =========================================================================

sku_marketplace = Table(
    'sku_marketplace', metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('sku_anymarket', String(100), nullable=True),
    Column('sku_marketplace', String(100), nullable=True),
    Column('numero_pedido', String(50), nullable=False),
    Column('valor_liquido', Float, nullable=True, default=0),
    Column('valor_bruto', Float, nullable=True, default=0),
    Column('marketplace_id', Integer, ForeignKey('marketplaces.id'), nullable=False),
    extend_existing=True
)

vendas = Table(
    'vendas', metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('sku_marketplace_id', Integer, ForeignKey('sku_marketplace.id'), nullable=False),
    Column('data', Date, nullable=False),
    Column('valor_liquido', Float, nullable=True),
    Column('valor_frete', Float, nullable=True),
    Column('status', String(50), nullable=True),
    extend_existing=True
)

arquivos_processados = Table(
    'arquivos_processados', metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('nome_arquivo', String(255), nullable=False, unique=True),
    Column('data_processo', Date, nullable=False, server_default=func.current_date()),
    Column('status', String(50), nullable=False),
    Column('observacoes', String, nullable=True),
    extend_existing=True
)

comissoes_periodo = Table(
    'comissoes_periodo', metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('data_inicio', Date, nullable=False),
    Column('data_fim', Date, nullable=False),
    Column('porcentagem', Numeric(5,4), nullable=False),
    Column('sku_marketplace', String(255), nullable=True),  # SKU específico ou NULL
    Column('marketplace_id', Integer, ForeignKey('marketplaces.id'), nullable=True),
    extend_existing=True
)

comissoes_pedido = Table(
    'comissoes_pedido', metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('data', Date, nullable=False),
    Column('porcentagem', Numeric(5,4), nullable=False),
    Column('sku_marketplace_id', Integer, ForeignKey('sku_marketplace.id'), nullable=False),
    CheckConstraint("porcentagem >= 0 AND porcentagem <= 1", name='comissoes_pedido_porcentagem_check'),
    extend_existing=True
)

metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

# =========================================================================
# FUNÇÕES AUXILIARES
# =========================================================================

def get_marketplace_id(marketplace_name: str) -> int:
    """
    Retorna o ID do marketplace:
      - 5, se contiver "centauro"
      - 6, se contiver "netshoes"
    Lança exceção se não reconhecer.
    """
    mlower = marketplace_name.lower()
    if "centauro" in mlower:
        return 5
    elif "netshoes" in mlower:
        return 6
    else:
        raise ValueError(f"Marketplace não reconhecido: {marketplace_name}")

def buscar_comissao_periodo(data_pedido, sku):
    """
    Busca a porcentagem em comissoes_periodo para a data_pedido e SKU informados.
    Se não existir para aquele SKU, busca o registro com sku_marketplace IS NULL.
    Retorna None se não encontrar nada.
    """
    sql_union = text(f"""
        SELECT porcentagem
        FROM comissoes_periodo cp
        WHERE :data_pedido BETWEEN cp.data_inicio AND cp.data_fim
          AND cp.sku_marketplace = :sku
        
        UNION
        
        SELECT porcentagem
        FROM comissoes_periodo cp
        WHERE :data_pedido BETWEEN cp.data_inicio AND cp.data_fim
          AND cp.sku_marketplace IS NULL
          AND NOT EXISTS (
              SELECT 1
              FROM comissoes_periodo cp2
              WHERE :data_pedido BETWEEN cp2.data_inicio AND cp2.data_fim
                AND cp2.sku_marketplace = :sku
          )
    """)

    result = session.execute(
        sql_union,
        {"data_pedido": data_pedido, "sku": sku}
    ).fetchone()
    
    if result:
        return float(result[0])
    else:
        return None

def validar_porcentagem(p: float):
    if not (0 <= p <= 1):
        raise ValueError("Porcentagem de comissão fora do intervalo permitido (0-1).")

def inserir_comissoes_pedido(data_pedido, porcentagem_comissao, sku_marketplace_id):
    """
    Insere um registro na tabela comissoes_pedido.
    """
    try:
        porcentagem_comissao = round(porcentagem_comissao, 4)
        validar_porcentagem(porcentagem_comissao)

        ins = comissoes_pedido.insert().values(
            data=data_pedido,
            porcentagem=porcentagem_comissao,
            sku_marketplace_id=sku_marketplace_id
        )
        session.execute(ins)
        session.commit()
        logger.info(
            f"[COMISSÃO] Inserida para SKU {sku_marketplace_id}, {porcentagem_comissao*100:.2f}%."
        )
    except Exception as e:
        logger.error(f"Erro ao inserir comissão para SKU={sku_marketplace_id}: {e}")
        session.rollback()

def inserir_arquivos_processados(nome_arquivo, status, observacoes=None):
    """
    Registra o processamento do arquivo na tabela arquivos_processados.
    """
    try:
        data_proc = datetime.now().date()
        ins = arquivos_processados.insert().values(
            nome_arquivo=nome_arquivo,
            data_processo=data_proc,
            status=status,
            observacoes=observacoes
        )
        session.execute(ins)
        session.commit()
        logger.info(f"Registro inserido em 'arquivos_processados' para '{nome_arquivo}' = {status}")
    except Exception as e:
        logger.error(f"Erro ao inserir registro em 'arquivos_processados': {e}")
        session.rollback()

def upsert_sku_marketplace(
    pedido: str,
    sku_any: str,
    sku_mk: str,
    valor_bruto: float,
    marketplace_id: int
) -> int:
    """
    Verifica se 'pedido' já existe em sku_marketplace.
    Se existir, faz update. Senão, faz insert. Retorna o id.
    """
    try:
        sel = sku_marketplace.select().where(sku_marketplace.c.numero_pedido == pedido)
        existente = session.execute(sel).fetchone()
        
        if existente:
            sku_id = existente.id
            upd = (
                sku_marketplace.update()
                .where(sku_marketplace.c.id == sku_id)
                .values(
                    sku_anymarket=sku_any,
                    sku_marketplace=sku_mk,
                    valor_bruto=valor_bruto,
                    marketplace_id=marketplace_id
                )
            )
            session.execute(upd)
            session.commit()
            logger.info(f"[SKU] Atualizado ID={sku_id} - pedido='{pedido}'.")
        else:
            ins = sku_marketplace.insert().values(
                numero_pedido=pedido,
                sku_anymarket=sku_any,
                sku_marketplace=sku_mk,
                valor_bruto=valor_bruto,
                marketplace_id=marketplace_id
            )
            result = session.execute(ins)
            session.commit()
            sku_id = result.inserted_primary_key[0]
            logger.info(f"[SKU] Inserido novo ID={sku_id} - pedido='{pedido}'.")

        return sku_id
    except Exception as e:
        logger.error(f"Erro no upsert_sku_marketplace para pedido='{pedido}': {e}")
        session.rollback()
        return None

def inserir_venda(
    sku_id: int,
    data_pedido: datetime.date,
    valor_liquido: float,
    valor_frete: float,
    status_pedido: str
):
    """
    Cria um registro na tabela vendas referenciando o 'sku_marketplace_id'.
    """
    try:
        ins = vendas.insert().values(
            sku_marketplace_id=sku_id,
            data=data_pedido,
            valor_liquido=valor_liquido,
            valor_frete=valor_frete,
            status=status_pedido
        )
        session.execute(ins)
        session.commit()
        logger.info(f"[VENDAS] Registrada venda SKU={sku_id}, data={data_pedido}.")
    except Exception as e:
        logger.error(f"Erro ao inserir venda SKU={sku_id}: {e}")
        session.rollback()

# ALTERADO: Removemos o "try/except" que envolvia todo o loop e passamos a usar um 'for' + 'try/except' interno
def processar_xlsx(file_path, reprocessar=True):
    """
    Lê o XLSX, insere/atualiza 'sku_marketplace', insere em 'vendas' e,
    se houver comissão, insere em 'comissoes_pedido'.

    Se reprocessar=True, não cria novamente pedidos que já estejam tanto em
    'sku_marketplace' quanto em 'vendas'.

    Agora, caso uma linha apresente erro, o processamento continua
    (não interrompe todo o arquivo).
    """
    nome_arquivo = os.path.basename(file_path)
    linhas_com_erro = 0

    try:
        df = pd.read_excel(file_path, decimal=',')
        logger.info(f"Lendo '{nome_arquivo}'. Total de registros: {len(df)}")
    except Exception as e:
        logger.error(f"Erro ao ler o arquivo '{nome_arquivo}': {e}")
        inserir_arquivos_processados(nome_arquivo, status='ERRO', observacoes=str(e))
        return

    for index, row in df.iterrows():
        try:
            pedido = str(row["CÓDIGO PEDIDO"])
            data_pedido = pd.to_datetime(row["DATA PEDIDO"], dayfirst=True).date()
            sku_any = str(row["SKU PRODUTO"])
            sku_mk = str(row["SKU DO PRODUTO NO MARKETPLACE"])
            valor_bruto = float(row["TOTAL DO PEDIDO"])
            valor_liquido = float(row["VALOR TOTAL DOS PRODUTOS"])
            valor_frete = float(row["FRETE"])
            status_pedido = str(row["STATUS"]).strip()
            marketplace_str = str(row["MARKETPLACE"])

            # Se o status for Cancelado ou Pendente, pula sem inserir no banco
            if status_pedido.lower() in ["cancelado", "pendente"]:
                logger.info(f"Pedido '{pedido}' com status '{status_pedido}' não será inserido.")
                continue

            mk_id = get_marketplace_id(marketplace_str)

            if reprocessar:
                # Verifica se o pedido já existe em sku_marketplace
                sel_sku = sku_marketplace.select().where(sku_marketplace.c.numero_pedido == pedido)
                existente_sku = session.execute(sel_sku).fetchone()

                if existente_sku:
                    sku_id_ex = existente_sku.id
                    # Verifica se já existe venda para este sku_marketplace_id
                    sel_venda = vendas.select().where(vendas.c.sku_marketplace_id == sku_id_ex)
                    existente_venda = session.execute(sel_venda).fetchone()
                    if existente_venda:
                        logger.info(f"Reprocessamento: pedido '{pedido}' já possui venda registrada. Pulando...")
                        continue

            sku_id = upsert_sku_marketplace(
                pedido=pedido,
                sku_any=sku_any,
                sku_mk=sku_mk,
                valor_bruto=valor_bruto,
                marketplace_id=mk_id
            )

            if sku_id:
                inserir_venda(
                    sku_id=sku_id,
                    data_pedido=data_pedido,
                    valor_liquido=valor_liquido,
                    valor_frete=valor_frete,
                    status_pedido=status_pedido
                )

                comissao_periodo = buscar_comissao_periodo(data_pedido, pedido)
                if comissao_periodo is not None:
                    inserir_comissoes_pedido(
                        data_pedido=data_pedido,
                        porcentagem_comissao=comissao_periodo,
                        sku_marketplace_id=sku_id
                    )
                else:
                    logger.info(f"Nenhuma comissão encontrada para o pedido '{pedido}' na data {data_pedido}.")

        except Exception as e_linha:
            linhas_com_erro += 1
            logger.error(f"Erro ao processar a linha {index} (pedido='{row.get('CÓDIGO PEDIDO', 'N/A')}'): {e_linha}")
            session.rollback()
            continue

    if linhas_com_erro > 0:
        inserir_arquivos_processados(
            nome_arquivo,
            status='PROCESSADO COM ERROS',
            observacoes=f"Houveram {linhas_com_erro} linha(s) com erro no processamento."
        )
    else:
        inserir_arquivos_processados(
            nome_arquivo,
            status='PROCESSADO',
            observacoes="Processado sem erros."
        )

    """
    Lê o XLSX, insere/atualiza 'sku_marketplace', insere em 'vendas' e,
    se houver comissão, insere em 'comissoes_pedido'.

    Se reprocessar=True, não cria novamente pedidos que já estejam tanto em
    'sku_marketplace' quanto em 'vendas'.

    Agora, caso uma linha apresente erro, o processamento continua
    (não interrompe todo o arquivo).
    """
    nome_arquivo = os.path.basename(file_path)
    
    # Usaremos esta variável para contar quantas linhas falharam.
    linhas_com_erro = 0

    try:
        df = pd.read_excel(file_path, decimal=',')
        logger.info(f"Lendo '{nome_arquivo}'. Total de registros: {len(df)}")
    except Exception as e:
        # Se der erro ao abrir/converter o arquivo (ex.: planilha corrompida),
        # encerramos aqui pois não conseguimos nem iterar.
        logger.error(f"Erro ao ler o arquivo '{nome_arquivo}': {e}")
        inserir_arquivos_processados(nome_arquivo, status='ERRO', observacoes=str(e))
        return

    # Se chegou até aqui, conseguimos ler o DataFrame com sucesso.
    for index, row in df.iterrows():
        try:
            pedido = str(row["CÓDIGO PEDIDO"])
            data_pedido = pd.to_datetime(row["DATA PEDIDO"], dayfirst=True).date()
            sku_any = str(row["SKU PRODUTO"])
            sku_mk = str(row["SKU DO PRODUTO NO MARKETPLACE"])
            valor_bruto = float(row["TOTAL DO PEDIDO"])
            valor_liquido = float(row["VALOR TOTAL DOS PRODUTOS"])
            valor_frete = float(row["FRETE"])
            status_pedido = str(row["STATUS"]).strip()
            marketplace_str = str(row["MARKETPLACE"])

            # Se o status for Cancelado ou Pendente, pula sem inserir no banco
            if status_pedido.lower() in ["cancelado", "pendente"]:
                logger.info(f"Pedido '{pedido}' com status '{status_pedido}' não será inserido.")
                continue

            mk_id = get_marketplace_id(marketplace_str)

            if reprocessar:
                # Verifica se pedido já existe em sku_marketplace
                sel_sku = sku_marketplace.select().where(sku_marketplace.c.numero_pedido == pedido)
                existente_sku = session.execute(sel_sku).fetchone()

                if existente_sku:
                    sku_id_ex = existente_sku.id
                    # Verifica se existe venda para este sku_marketplace_id
                    sel_venda = vendas.select().where(vendas.c.sku_marketplace_id == sku_id_ex)
                    existente_venda = session.execute(sel_venda).fetchone()

                    if existente_venda:
                        logger.info(
                            f"Reprocessamento: pedido '{pedido}' já tem venda criada. Pulando..."
                        )
                        continue

            sku_id = upsert_sku_marketplace(
                pedido=pedido,
                sku_any=sku_any,
                sku_mk=sku_mk,
                valor_bruto=valor_bruto,
                marketplace_id=mk_id
            )

            if sku_id:
                inserir_venda(
                    sku_id=sku_id,
                    data_pedido=data_pedido,
                    valor_liquido=valor_liquido,
                    valor_frete=valor_frete,
                    status_pedido=status_pedido
                )

                comissao_periodo = buscar_comissao_periodo(data_pedido, pedido)
                if comissao_periodo is not None:
                    inserir_comissoes_pedido(
                        data_pedido=data_pedido,
                        porcentagem_comissao=comissao_periodo,
                        sku_marketplace_id=sku_id
                    )
                else:
                    logger.info(
                        f"Nenhuma comissão encontrada p/ pedido='{pedido}' na data {data_pedido}."
                    )

        except Exception as e_linha:
            # ALTERADO: se der erro em qualquer parte do processamento da linha,
            # apenas logamos e continuamos. 
            linhas_com_erro += 1
            logger.error(
                f"Erro ao processar a linha {index} (pedido='{row.get('CÓDIGO PEDIDO', 'N/A')}'): {e_linha}"
            )
            # Fazemos rollback para garantir que nada parcial fique no banco:
            session.rollback()
            # continua para a próxima linha
            continue

    # Ao final do loop, vemos se teve ou não linhas com erro
    if linhas_com_erro > 0:
        # Você pode escolher marcar como "PROCESSADO COM ERROS"
        inserir_arquivos_processados(
            nome_arquivo,
            status='PROCESSADO COM ERROS',
            observacoes=f"Houveram {linhas_com_erro} linha(s) com erro no processamento."
        )
    else:
        inserir_arquivos_processados(
            nome_arquivo,
            status='PROCESSADO',
            observacoes="Processado sem erros."
        )

def main():
    base_dir = os.getcwd()
    folder_path_vendas = os.path.join(base_dir, "Vendas")

    if not os.path.isdir(folder_path_vendas):
        logger.error(f"Pasta '{folder_path_vendas}' não encontrada.")
        inserir_arquivos_processados("Vendas", status="ERRO", observacoes="Pasta não encontrada.")
        return
    
    for filename in os.listdir(folder_path_vendas):
        if filename.startswith('~$'):
            logger.info(f"Ignorando arquivo temporário: {filename}")
            continue

        if filename.lower().endswith(".xlsx"):
            file_path = os.path.join(folder_path_vendas, filename)
            if not os.path.exists(file_path):
                logger.error(f"Arquivo '{file_path}' não encontrado.")
                inserir_arquivos_processados(filename, status="ERRO", observacoes="Arquivo não encontrado.")
                continue

            logger.info(f"Processando arquivo (exemplo como primeira vez): {file_path}")
            processar_xlsx(file_path, reprocessar=True)
        else:
            logger.info(f"Ignorando arquivo que não é XLSX: {filename}")

if __name__ == "__main__":
    main()


Erro ao inserir comissão para SKU=8084: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "comissoes_dia_2024_sku_marketplace_id_data_key"
DETAIL:  Key (sku_marketplace_id, data)=(8084, 2024-07-31) already exists.

[SQL: INSERT INTO comissoes_pedido (data, porcentagem, sku_marketplace_id) VALUES (%(data)s, %(porcentagem)s, %(sku_marketplace_id)s) RETURNING comissoes_pedido.id]
[parameters: {'data': datetime.date(2024, 7, 31), 'porcentagem': 0.21, 'sku_marketplace_id': 8084}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
Erro ao inserir registro em 'arquivos_processados': (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "arquivos_processados_nome_arquivo_key"
DETAIL:  Key (nome_arquivo)=(2024-3T Vendas Anymarket Pipe Centauro e Netshoes.xlsx) already exists.

[SQL: INSERT INTO arquivos_processados (nome_arquivo, data_processo, status, observacoes) VALUES (%(nome_arquivo)s, %(data_processo)s, %(status)s, %(obse